<a href="https://colab.research.google.com/github/mingd00/aivle-miniproj6/blob/main/2%EC%B0%A8.%20%EC%9D%91%EA%B8%89%EC%83%81%ED%99%A9%20%EC%9D%B8%EC%8B%9D%20%EB%B0%8F%20%EC%9D%91%EA%B8%89%EC%8B%A4%20%EC%97%B0%EA%B3%84/4-1.%20%EB%AA%A8%EB%93%88%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **응급상황 자동 인식 및 응급실 연계 서비스**
### **모듈화**

## **1.환경설정**

- 구글 드라이브 연결

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
path = '/content/drive/MyDrive/project6_2/'

## **2.모듈 구성**

In [7]:
%%writefile /content/drive/MyDrive/project6_2/emergency.py

import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import openai
from openai import OpenAI
import json
import torch

# 0. load key file------------------
def load_keys(path):
  with open(path, 'r') as file:
    return file.readline().strip()


# 1-1 audio2text--------------------
def audio2text(audio_path, filename):
    client = OpenAI()

    audio_file = open(audio_path + filename, "rb")
    transcript = client.audio.transcriptions.create(
        file=audio_file,
        model="whisper-1",
        language="ko",
        response_format="text",
    )

    return transcript


# 1-2 text2summary------------------
def text2summary(system_role, input_text):
    client = OpenAI()
    system_role = '당신은 입력된 텍스트를 가장 간결하고 핵심적인 표현으로 요약하는 어시스턴트입니다. 불필요한 설명을 생략하고 핵심적인 정보를 짧은 문장으로 요약하세요.'

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": system_role
            },
            {
                "role": "user",
                "content": f'"{input_text}" 어떤 상황인지 알 수 있게 문장자체를 요약해줘'
            }
        ]
    )

    return response.choices[0].message.content


# 2. model prediction------------------
def predict(text, model, tokenizer, device):
    # 입력 문장 토크나이징
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # 각 텐서를 GPU로 이동

    # 모델 예측
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)

    # 로짓을 소프트맥스로 변환하여 확률 계산
    logits = outputs.logits
    probabilities = logits.softmax(dim=1)

    # 가장 높은 확률을 가진 클래스 선택
    pred = torch.argmax(probabilities, dim=-1).item()

    return pred, probabilities


# 3-1. get_distance------------------
def get_dist(start_lat, start_lng, dest_lat, dest_lng, c_id, c_key):
    url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": c_id,
        "X-NCP-APIGW-API-KEY": c_key,
    }
    params = {
        "start": f"{start_lng},{start_lat}",  # 출발지 (경도, 위도)
        "goal": f"{dest_lng},{dest_lat}",    # 목적지 (경도, 위도)
        "option": "trafast"  # 실시간 빠른 길 옵션
    }

    # 요청하고, 답변 받아오기
    response = requests.get(url, params, headers=headers)
    response = response.json()

    if response['code'] == 1:
        dist = 0
    elif response['code'] == 0:
        # 'route' 키가 있는지 확인 후 접근
        if 'route' in response and 'trafast' in response['route']:
            dist = response['route']['trafast'][0]['summary']['distance']  # m(미터)
        else:
            print("No route found in the response.")
            dist = None  # 또는 기본값을 반환할 수 있음
    else:
        print(f"Unexpected response code: {response['code']}")
        dist = None

    return dist


# 3-2. recommendation------------------
def recommend_hospital(data, lat, lng, range, c_id, c_key) :
  li = []

  # 특정 범위 데이터만 추출
  filtered_data = data[data['위도'].between(lat - range, lat + range) & data['경도'].between(lng - range, lng + range)].copy()

  # 각 병원까지의 거리를 계산하여 컬럼 생성
  for d in filtered_data.iterrows():
    li.append([d[1]['병원이름'], get_dist(lat, lng, d[1]['위도'], d[1]['경도'], c_id, c_key)])

  # 상위 3개 추출
  li = sorted(li, key=lambda x: (x[1]))[:3]
  return li

Overwriting /content/drive/MyDrive/project6_2/emergency.py
